In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F # All functions that don't have any parameters
import numpy as np
from  boxScore  import boxScore
from sklearn.model_selection import KFold

from torch.utils.data import DataLoader # data management 
# 
from NeuralNetwork import NN
from DatasetPrivate import MyDataset
import numpy as np


import winsound
import wandb

# Preparation of Data

In [11]:
years="2019-20"
# stats="traditional"
stats='advance'
box_score=boxScore(years,stats)

x_train, x_test, y_train, y_test=box_score.separation()
y_train=np.array(y_train)
y_test=np.array(y_test)

input_size=len(x_train.columns)
activation=['relu','sigmoid']
number_neurons=[10,30,50,100,150,200]
# number_neurons=[10,30]
possible_learning_rate=[0.0001,0.001,0.01]
# possible_learning_rate=[0.0001]
num_epochs=500
batch_size=150
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_LL=DataLoader(MyDataset(x_train,y_train),batch_size=batch_size,shuffle=False)
test_LL=DataLoader(MyDataset(x_test,y_test),batch_size=batch_size,shuffle=False)
kf = KFold(n_splits=10, random_state=1, shuffle=True)

# First 

In [12]:
best_config_array=[]
best_history_array=[]
for num in number_neurons:
    best_config_tmp={'val_acc':0}
    best_history_tmp=[]
    for el in possible_learning_rate:
        for act in activation:
                
                # Initialize model
                model = NN(input_size=input_size,size_hidden_level=num,act=act,learning_rate=el).to(device)

                
                
                config={
                    "learning_rate":el,
                    'num_neurons':num,
                    'acti_fun':act,
                    "batch_size": batch_size,
                    "epochs": num_epochs,
                    "architecture": "OneHiddenLayer",
                    "dataset":years+"_"+stats
                }
                
                best_config_tmp,best_history_tmp=model.MyTrain(kf=kf,config=config,best_config_tmp=best_config_tmp,
                best_history_tmp=best_history_tmp,train_LL=train_LL,test_LL=test_LL,device=device)
               
                
                    
    print("Best model",best_config_tmp)
    best_config_array.append(best_config_tmp)
    best_history_array.append(best_history_tmp)
    
# Doing the mean between the 10 fold
for num,array in  enumerate(best_history_array):
    tmp=[ torch.mean(array[0],axis=0),torch.mean(array[1],axis=0),torch.mean(array[2],axis=0),torch.mean(array[3],axis=0)]
  
    best_history_array[num]=tmp

# winsound.Beep(440,2500)

Best model {'learning_rate': 0.01, 'num_neurons': 10, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2019-20_advance', 'val_acc': 0.6539999842643738}
Best model {'learning_rate': 0.01, 'num_neurons': 30, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2019-20_advance', 'val_acc': 0.6620000004768372}
Best model {'learning_rate': 0.01, 'num_neurons': 50, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2019-20_advance', 'val_acc': 0.6760000586509705}
Best model {'learning_rate': 0.01, 'num_neurons': 100, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2019-20_advance', 'val_acc': 0.6800000071525574}
Best model {'learning_rate': 0.01, 'num_neurons': 150, 'acti_fun': 'sigmoid', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2019-20_advance', 'val_acc': 

In [ ]:
winsound.Beep(440,2500)

In [13]:
def wandbWrite(project_name,best_config_array,best_history_array,name_runs):
    for model_history,config in zip(best_history_array,best_config_array):
        
        

        run = wandb.init(project=project_name, config=config)
        name=name_runs+str(config["num_neurons"])
        run.name=name
        # For not repeat two times the val_acc value in the config paper
        config.pop("val_acc")
        config = wandb.config
        for epoch in range (len(model_history[0])):
            wandb.log({'epochs': epoch,
                'loss': round( model_history[0][epoch].item(),3),
                'acc': round(model_history[1][epoch].item(),3), 
                'val_loss': round(model_history[2][epoch].item(),3),
                'val_acc':round(model_history[3][epoch].item(),3)
                }
                )
        
        run.finish()

In [14]:
wandb.login()



project_name="T"+stats[0:3].capitalize()+years[2:].replace("-","")+"runs"
wandbWrite(project_name,best_config_array,best_history_array,"onehidden_")


acc,▁▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇███████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁
val_acc,▅▃▃▄▁▃▃▄▅▅▆▄▆▄▄▆▃▆▅▅▅▆▃▅▅█▆▆▄▄▃▆▄▅▆▅▆▃▅▅
val_loss,▁▁▁▂▂▂▂▂▃▂▂▂▂▃▄▂▄▄▄▄▃▄▄▄▆▅▄▆▅▆▆▅▆▆▅▇▆█▆▇
acc,0.86
epochs,499
loss,0.142
val_acc,0.654
val_loss,1.122


acc,▁▂▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_acc,▄▂▂▁▃▄▅▄▅▄▇▄▅▄▅▄▇▅█▄▅▅▄▅▅▂▄▅▆▆▅▅▅▆▆▄▄▄▄▅
val_loss,▁▂▁▃▂▃▃▃▂▄▃▄▃▄▄▄▄▅▄▅▅▅▅▄▄▆▆▅▆▅▆▇▆▆▆▇█▇█▇
acc,0.86
epochs,499
loss,0.143
val_acc,0.662
val_loss,1.084


acc,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇██▇██▇████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁
val_acc,▁▂▃▂▁▂▂▃▃▂▃▃▂▄▃▄▄▅▄▅▅▄▅▄▄▅▅▅▅▅▅▆▅▆▅▆▇▇▇█
val_loss,▁▃▃▂▂▃▃▃▃▃▄▄▃▃▄▄▃▄▅▄▅▅▅▅▅▅▆▇▇▆▆▆▇▇▇▇▆▇█▆
acc,0.859
epochs,499
loss,0.14
val_acc,0.676
val_loss,0.854


acc,▁▅▅▇▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇██▇▇▇█▇████████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
val_acc,▃▃▁▄▄▆▅▆▄▆▇▇▆▆▅▇▇▆▇▅▅▄▄▆█▆▇▇▇█▆▆▇▇█▇▇▇▇█
val_loss,▂▂▂▂▃▁▃▂▃▄▃▂▄▃▄▃▄▅▃▅▄▅▇▅▃▅▆▅▆▅▆▇▆▇▆▇▇▇▇█
acc,0.861
epochs,499
loss,0.127
val_acc,0.68
val_loss,1.052


acc,▁▃▄▆▅▃▄█▄▄▄▃▄▆▇▆▆▇▇▆██▇▇▇█▅▇▆▇█▅▇▇█▇████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▂▁▁
val_acc,▂▄▅▆▇▁▂▇▂▄▅▁█▄▅▃▆▄█▄█▄▄▇▄▇▇▇▅▇█▆▆▅▇▆▇▇▅█
val_loss,▃▂▁▃▂▂▂▄▃▁▃▆▂▃▃▄▃▄▆▃▃▃▄▄▄▄▃▇▇▆▇▇▆▇█▇▆█▇█
acc,0.837
epochs,499
loss,0.322
val_acc,0.67
val_loss,0.654


acc,▁▁▂▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁
val_acc,▁▂▂▂▃▆▇▆▇▆▇▆▆▇▇▇▆▇▇▇▇▇█▇▆▇█▇▇█▇▇▇▇▇▆▆▆▇▇
val_loss,▇█▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁▁
acc,0.817
epochs,499
loss,0.427
val_acc,0.702
val_loss,0.62


<!-- Best model {'learning_rate': 0.001, 'num_neurons': 10, 'acti_fun': 'relu', 'loss': 0.0193, 'acc': 0.9987, 'val_loss': 1.4627, 'val_acc': 0.6}
Best model {'learning_rate': 0.0001, 'num_neurons': 30, 'acti_fun': 'sigmoid', 'loss': 0.4884, 'acc': 0.9517, 'val_loss': 0.6466, 'val_acc': 0.64}
Best model {'learning_rate': 0.0001, 'num_neurons': 50, 'acti_fun': 'sigmoid', 'loss': 0.5163, 'acc': 0.9437, 'val_loss': 0.656, 'val_acc': 0.66}
Best model {'learning_rate': 0.0001, 'num_neurons': 100, 'acti_fun': 'relu', 'loss': 0.0628, 'acc': 0.9951, 'val_loss': 0.775, 'val_acc': 0.64}
Best model {'learning_rate': 0.001, 'num_neurons': 150, 'acti_fun': 'relu', 'loss': 0.0135, 'acc': 0.9998, 'val_loss': 1.4478, 'val_acc': 0.64}
Best model {'learning_rate': 0.0001, 'num_neurons': 200, 'acti_fun': 'relu', 'loss': 0.0487, 'acc': 1.0, 'val_loss': 0.8261, 'val_acc': 0.64} -->

# L2 Regolarization


In [15]:
best_config_array=[]
best_history_array=[]
possible_pL2=[0.0001,0.001,0.01]
for num in number_neurons:
    best_config_tmp={'val_acc':0}
    best_history_tmp=[]
    for el in possible_learning_rate:
        for act in activation:
            for pl2 in possible_pL2:
            # Initialize model
                model = NN(input_size=input_size,size_hidden_level=num,act=act,learning_rate=el,pl2=pl2).to(device)

                
                
                config={
                    "learning_rate":el,
                    'num_neurons':num,
                    'acti_fun':act,
                    "batch_size": batch_size,
                    "epochs": num_epochs,
                    "pl2":pl2,
                    "architecture": "OneHiddenLayerL2",
                    "dataset":years+"_"+stats
                }
                
                best_config_tmp,best_history_tmp=model.MyTrain(kf=kf,config=config,best_config_tmp=best_config_tmp,
                best_history_tmp=best_history_tmp,train_LL=train_LL,test_LL=test_LL,device=device)
            
                    
    print("Best model",best_config_tmp)
    best_config_array.append(best_config_tmp)
    best_history_array.append(best_history_tmp)
    
# Doing the mean between the 10 fold
for num,array in  enumerate(best_history_array):
    tmp=[ torch.mean(array[0],axis=0),torch.mean(array[1],axis=0),torch.mean(array[2],axis=0),torch.mean(array[3],axis=0)]
  
    best_history_array[num]=tmp


Best model {'learning_rate': 0.01, 'num_neurons': 10, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'pl2': 0.0001, 'architecture': 'OneHiddenLayerL2', 'dataset': '2019-20_advance', 'val_acc': 0.6880000233650208}
Best model {'learning_rate': 0.01, 'num_neurons': 30, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'pl2': 0.01, 'architecture': 'OneHiddenLayerL2', 'dataset': '2019-20_advance', 'val_acc': 0.6899999976158142}
Best model {'learning_rate': 0.01, 'num_neurons': 50, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'pl2': 0.0001, 'architecture': 'OneHiddenLayerL2', 'dataset': '2019-20_advance', 'val_acc': 0.6880000233650208}
Best model {'learning_rate': 0.01, 'num_neurons': 100, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'pl2': 0.01, 'architecture': 'OneHiddenLayerL2', 'dataset': '2019-20_advance', 'val_acc': 0.6859999895095825}
Best model {'learning_rate': 0.01, 'num_neurons': 150, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'pl2': 0.01, '

In [ ]:
winsound.Beep(440,2500)

In [16]:
wandb.login()

project_name="T"+stats[0:3].capitalize()+years[2:].replace("-","")+"runs"
wandbWrite(project_name,best_config_array,best_history_array,"onehiddenL2_")


acc,▁▁▂▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_acc,▃▁▃▆▄▅▄▆▄▇▆▆▆▆▇▆▇▅▄▆▆▆▅▇▆▇▆▇▇▇▇▇▇▆▆▆▄▇█▆
val_loss,▁▂▁▁▃▃▃▃▄▄▂▂▂▃▄▃▄▅▆▅▅▅▅▅▅▄▆▅▄▅▅▇▆▇█▅▇▇▇▇
acc,0.859
epochs,499
loss,0.152
val_acc,0.688
val_loss,0.958


acc,▁▁▂▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_acc,▃▅▂▄▃▃▃▃▁▁▁▂▂▃▁▃▃▄▄▄▃▃▃▅▅▅▆▅▆▅█▆████▇▆▇▇
val_loss,▃▃▃▅▂▄▁▄▃▃▂▂▅█▄▄▄▄▄▆▅▃▂▅▄▇▂▆▄▄▄▄▆▄▄▆▅▄▄▃
acc,0.855
epochs,499
loss,0.203
val_acc,0.69
val_loss,0.684


acc,▁▁▃▄▄▅▆▆▆▆▇▇▇▇▇▇▇██▇██▇▇▇█▇█▇█▇█████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
val_acc,▁▂▂▂▃▃▃▃▂▄▅▆▆▆▅▆▅▆▇▆▇▆▇▇▇▆▇▇▇▆▆▇▇▇█▇▆█▇▇
val_loss,▁▂▂▁▂▁▃▃▂▂▂▃▃▃▃▅▃▅▄▄▅▆▄▄▅▅▅▅▅▅▆▇▆▆▆▆▇▆█▆
acc,0.861
epochs,499
loss,0.133
val_acc,0.688
val_loss,0.969


acc,▁▂▄▄▅▅▆▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇████████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▇▆▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
val_acc,▃▁▂▆▄▅▄▄▆▇▆▇▇▇▇▇▇▇▆▇▇▇█▇▆▆█▇▇▆▇▇▇▇▇▆▆▇▆▇
val_loss,▂▁▆▄▄▄▂▃▅▇▇▆▆▆▅▅▂▇▇▅▃▅▆▄█▇▆▃▃▃█▂▄▆▇█▆▅▅█
acc,0.852
epochs,499
loss,0.196
val_acc,0.686
val_loss,0.7


acc,▁▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇██▇▇██████████████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_acc,▁▅▅▅▃▆▅▆▇▇▆▇▆▆▆▇▇▆▇▇▆▇▆▆▆▅█▇▆██▆▆▇▇▇▆▇██
val_loss,▄▂▃▃▂▆▃▆▁▃▃▅▃█▃▂▆▃▅▃▄▄▇▇▃▅▇▄▅▄▇▅▅█▆▆▅▅▇▅
acc,0.855
epochs,499
loss,0.188
val_acc,0.69
val_loss,0.691


acc,▁▅▆▆▆▇▇▆▇▇▆▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█████▇██▇█████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
val_acc,▁▄▂▂▅▃▂▂▄▂▄▃▁▂▄▄▄▂▄▃▅▅▅▄▅▆▅▄▃▆▆▅▅▄▃▄█▄▅▅
val_loss,▂▆▂▂▃▅▃▂▂▃▄▃▂▃▃▄▄▁▄▃▃▂▃▃▃▄▄▃▅▂▅▄▅▄▄▆▅█▄▆
acc,0.855
epochs,499
loss,0.186
val_acc,0.686
val_loss,0.679


# Dropout

In [17]:
best_config_array=[]
best_history_array=[]
possible_p=[0.5,0.6,0.65]
for num in number_neurons:
    best_config_tmp={'val_acc':0}
    best_history_tmp=[]
    for el in possible_learning_rate:
        for act in activation:
            for p in possible_p:
            # Initialize model
                model = NN(input_size=input_size,size_hidden_level=num,act=act,learning_rate=el,dropout=p).to(device)

                
                
                config={
                    "learning_rate":el,
                    'num_neurons':num,
                    'acti_fun':act,
                    "batch_size": batch_size,
                    "epochs": num_epochs,
                    "drop_rate":p,
                    "architecture": "OneHiddenLayerDropout",
                    "dataset":years+"_"+stats
                }
                
                best_config_tmp,best_history_tmp=model.MyTrain(kf=kf,config=config,best_config_tmp=best_config_tmp,
                best_history_tmp=best_history_tmp,train_LL=train_LL,test_LL=test_LL,device=device)
            
                
                    
    print("Best model",best_config_tmp)
    best_config_array.append(best_config_tmp)
    best_history_array.append(best_history_tmp)
    
# Doing the mean between the 10 fold
for num,array in  enumerate(best_history_array):
    tmp=[ torch.mean(array[0],axis=0),torch.mean(array[1],axis=0),torch.mean(array[2],axis=0),torch.mean(array[3],axis=0)]
  
    best_history_array[num]=tmp



Best model {'learning_rate': 0.01, 'num_neurons': 10, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'drop_rate': 0.65, 'architecture': 'OneHiddenLayerDropout', 'dataset': '2019-20_advance', 'val_acc': 0.6819999814033508}
Best model {'learning_rate': 0.01, 'num_neurons': 30, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'drop_rate': 0.6, 'architecture': 'OneHiddenLayerDropout', 'dataset': '2019-20_advance', 'val_acc': 0.6859999895095825}
Best model {'learning_rate': 0.01, 'num_neurons': 50, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'drop_rate': 0.5, 'architecture': 'OneHiddenLayerDropout', 'dataset': '2019-20_advance', 'val_acc': 0.6800000071525574}
Best model {'learning_rate': 0.01, 'num_neurons': 100, 'acti_fun': 'relu', 'batch_size': 150, 'epochs': 500, 'drop_rate': 0.6, 'architecture': 'OneHiddenLayerDropout', 'dataset': '2019-20_advance', 'val_acc': 0.6859999895095825}
Best model {'learning_rate': 0.01, 'num_neurons': 150, 'acti_fun': 'relu', 'batch_size

In [18]:
winsound.Beep(440,2500)

In [19]:
wandb.login()

project_name="T"+stats[0:3].capitalize()+years[2:].replace("-","")+"runs"

wandbWrite(project_name,best_config_array,best_history_array,"onehiddenDrop_")

acc,▃▁▂▂▂▃▃▃▃▃▅▅▄▅▅▅▅▅▄▆▆▅▄▅▅▆▆▆▅▇▆▆▇▇▆▆█▆▇▇
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▇███▇▇▇▇▇▆▅▆▆▅▆▅▅▅▅▅▄▅▄▅▃▃▃▄▄▂▃▄▃▂▃▃▁▃▁▂
val_acc,▃▁▃▂▃▂▁▂▂▄▃▄▃▂▅▅▅▇█▇██▇▇▇▇▆▅▆▇▇▇▆▇▆▅▆▆▇▆
val_loss,▁▁▂▃▃▃▃▂▃▃▂▅▃▂▃▃▄▅▄▆▃▅▅▃▅▅▄▅▅▅▅▅▆▆▅▆▅▆▆█
acc,0.766
epochs,499
loss,0.335
val_acc,0.682
val_loss,0.808


acc,▁▁▂▁▁▂▂▃▃▂▃▃▄▄▃▃▄▅▄▅▅▆▅▆▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇███▇▇▆▆▆▇▆▆▆▆▆▅▅▅▄▅▄▄▄▄▄▃▄▃▃▂▃▂▂▂▁▂▂▂▁
val_acc,▃▂▄▅▅▁▂▅▆▄▆▃▃▄▄▆▄▆▅▆▅▆▃▆▆▆▃▄▆▇▄▅▆▆█▆▄▇▇▆
val_loss,▁▃▂▂▁▂▄▃▂▁▂▄▃▄▃▂▄▃▄▄▄▃▇▄▅▆▆▆▆▄▆▅▇▆▆▆█▇▇█
acc,0.828
epochs,499
loss,0.217
val_acc,0.686
val_loss,0.825


acc,▁▂▁▂▃▃▄▄▄▃▄▄▅▅▅▅▅▅▆▅▅▆▆▇▇▆▇▇▇▆▇█▇▇█▇████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇██▇▇▆▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_acc,▄▁▂▄▅▁▄▄▅▇▅▄▅▆▅▇▆▇▇▄▇▇▆▆▅▅█▆▇▇█▇▇▆▆▅▆▃█▆
val_loss,▁▂▃▁▁▃▁▂▂▃▂▃▃▄▃▃▃▄▄▅▄▅▆▅▆▆▆▇▆▆▅▅▆█▇▆▇█▇█
acc,0.849
epochs,499
loss,0.168
val_acc,0.68
val_loss,0.946


acc,▁▁▁▂▄▂▄▃▅▄▄▅▅▅▆▆▆▅▆▅▆▆▆▆▇▇▆▇▇▇▇▇▇▇█▇████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▇▆▇▆▆▆▆▅▆▅▅▅▅▅▅▄▄▄▃▃▃▄▃▃▃▂▂▃▂▂▂▂▁▁▂
val_acc,▂▁▂▃▁▃▃▄▄▂▄▃▄▁▅▄▄▄▅▅▆▆▆█▆▇▇▇█▆█▇▆▇█▆▅█▅▅
val_loss,▁▂▁▂▁▁▂▁▃▂▃▂▃▂▃▄▃▄▃▃▃▄▃▅▄▄▅▄▅▆▆▅▅▄▆▅▇▇▇█
acc,0.848
epochs,499
loss,0.169
val_acc,0.686
val_loss,0.845


acc,▁▁▂▃▂▃▃▄▄▅▄▅▅▆▅▆▆▆▆▆▆▇▇▇▇▇█▇█▇██▇█▇███▇█
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇█▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▃▃▂▂▂▂▂▁▁▁▁
val_acc,▁▂▄▅▃▅▄▃▃▄▄▃▆▆▆▆▆▅▄▅▅▅▆█▇▆█▇▅▇▆█▅▆▇▇▆▅▇▇
val_loss,▂▁▂▁▁▃▂▂▂▄▃▄▄▃▄▃▄▃▅▆▆▅▄▃▅▆▆▆█▆▇▆▇▇▇▆▇██▇
acc,0.849
epochs,499
loss,0.16
val_acc,0.698
val_loss,0.849


acc,▁▂▂▃▃▅▅▅▅▅▆▆▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇███▇▇▇█████
epochs,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_acc,▁▃▃▆▅▅▄▆▆▆▆▆▆▆▆▆▇▅▇█▆▇██▆▇▆▇▇▇▆▇▅▆█▇▇▆▆█
val_loss,▂▁▂▁▂▂▁▃▃▂▃▄▃▃▄▂▄▃▄▄▄▅▅▄▇▄▅▆▅▅▇▇▅▅▅▇▇█▆▇
acc,0.856
epochs,499
loss,0.146
val_acc,0.686
val_loss,0.862
